In [23]:
import redis
import json
from datetime import date
r = redis.Redis(
    host='redis',
    port=6379,
    db=0,
    decode_responses=True
)
r.ping()

True

In [4]:
def add_player(player_id, name, country, score):
    player_key = f"player:{player_id}"
    r.hset(player_key, mapping={
        "name": name,
        "country": country,
        "games_played": 0,
        "score": score
    })
    r.zadd("leaderboard", {player_id: score})

In [33]:
add_player(1, "paco", "españa", 12)

In [10]:
def update_score(player_id, points):
    player_key = f"player:{player_id}"
    new_score = r.hincrbyfloat(player_key, "score", points)
    games_played = r.hincrby(player_key, "games_played", 1)
    r.zadd("leaderboard", {player_id: new_score})

In [11]:
update_score(1, 32)

In [14]:
def player_info(player_id):
    player_key = f"player:{player_id}"
    player_data = r.hgetall(player_key)
    return player_data

In [15]:
player_info(1)

{'name': 'paco', 'country': 'españa', 'games_played': '1', 'score': '44'}

In [18]:
def show_top_players(n):
    leaderboard_key = "leaderboard"
    top_players = r.zrevrange(leaderboard_key, 0, n - 1, withscores=True)
    print(f"Top {n} jugadores del leaderboard:\n")
    print(top_players)

In [20]:
show_top_players(1)

Top 1 jugadores del leaderboard:

[('1', 44.0)]


In [54]:
def register_login(player_id):
    fecha = date.today()
    key = f"unique:players:{fecha}"
    r.pfadd(key, player_id)

In [51]:
register_login(1)

In [55]:
def count_unique_logins(date):
    logins = r.pfcount(date)
    return logins

In [53]:
count_unique_logins(2025-11-11)

0